In [172]:
%%capture
from glob import glob
!pip install pytorch-lightning
!pip install keras-tuner

In [173]:
%%capture
import os
!pip install mne
import mne
import numpy as np
import pandas
import matplotlib.pyplot as plt

In [174]:
path=glob('dataverse_files-2/*.edf')
len(path)

28

In [175]:
healthyPath=[i for i in path if 'h' in i.split("/")[1]]
PatientPath=[i for i in path if 's' in i.split("/")[1]]

In [176]:
def read_data(filepath):
    data=mne.io.read_raw_edf(filepath,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=1,h_freq=30)
    epoch=mne.make_fixed_length_epochs(data,duration=4,overlap=1)
    arr=epoch.get_data()
    return arr

In [177]:
%%capture
healthy_epochArr=[read_data(i) for i in healthyPath]
patient_epochArr=[read_data(i) for i in PatientPath]

In [178]:
#len(mne_objectHealthy)
print(len(healthy_epochArr[13]))

301


In [179]:
control_epoch_labels=[len(i)*[0] for i in healthy_epochArr]
patient_epoch_labels=[len(i)*[1] for i in patient_epochArr]

In [180]:
data_list=healthy_epochArr+patient_epochArr
label_list=control_epoch_labels+patient_epoch_labels

In [181]:
group_list=[[i]*len(j) for i,j in enumerate (data_list)]

In [182]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [183]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(9605, 1000, 19) (9605,) (9605,)


In [184]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    break;

In [185]:
train_features.shape

(7662, 1000, 19)

In [186]:
# import torch.nn as nn 
# import torch

In [187]:
import numpy as np
data_array=np.concatenate(data_list)
label_array=np.concatenate(label_list)
group_array=np.concatenate(group_list)
data_array=np.moveaxis(data_array,1,2)
print(data_array.shape,label_array.shape,group_array.shape)

(9605, 1000, 19) (9605,) (9605,)


In [188]:
# accuracy=[]
# for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
#     train_features,train_labels=data_array[train_index],label_array[train_index]
#     val_features,val_labels=data_array[val_index],label_array[val_index]
#     scaler=StandardScaler3D()
#     train_features=scaler.fit_transform(train_features)
#     val_features=scaler.transform(val_features)
#     train_features=np.moveaxis(train_features,1,2)
#     val_features=np.moveaxis(val_features,1,2)

#     break

In [189]:
# train_features = torch.Tensor(train_features)
# val_features = torch.Tensor(val_features)
# train_labels = torch.Tensor(train_labels)
# val_labels = torch.Tensor(val_labels)

In [190]:
len(val_features),len(val_labels)

(1943, 1943)

In [191]:
train_features.shape

(7662, 1000, 19)

In [192]:
from tensorflow.keras.layers import Input,Dense,concatenate,Flatten,GRU,Conv1D
from tensorflow.keras.models import Model

In [193]:
def block(input):
    conv1 = Conv1D(32, 2, strides=2,activation='relu',padding="same")(input)
    conv2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(input)
    conv3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(input)
    x = concatenate([conv1,conv2,conv3],axis=2)
    return x

In [194]:
input= Input(shape=(1000,19))
block1=block(input)
block2=block(block1)
block3=block(block2)

In [195]:
gru_out1 = GRU(32,activation='tanh',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='tanh')(gru_out)


In [196]:
predictions = Dense(1,activation='sigmoid')(gru_out4)
model = Model(inputs=input, outputs=predictions)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [197]:
train_features.shape

(7662, 1000, 19)

In [198]:
##model.fit(,epochs=10,batch_size=128,validation_data=(val_features,val_labels))

In [199]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
train_dataset = train_dataset.batch(128)  
val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels))
val_dataset = val_dataset.batch(128)  

In [200]:
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
60/60 [==============================] - 23s 331ms/step - loss: 0.9260 - accuracy: 0.7549 - val_loss: 2.9320 - val_accuracy: 0.3721
Epoch 2/10
60/60 [==============================] - 18s 305ms/step - loss: 1.3458 - accuracy: 0.5210 - val_loss: 1.7434 - val_accuracy: 0.3721
Epoch 3/10
60/60 [==============================] - 19s 310ms/step - loss: 1.0605 - accuracy: 0.4093 - val_loss: 1.4661 - val_accuracy: 0.3721
Epoch 4/10
60/60 [==============================] - 19s 311ms/step - loss: 0.9562 - accuracy: 0.4370 - val_loss: 1.3360 - val_accuracy: 0.3721
Epoch 5/10
60/60 [==============================] - 20s 327ms/step - loss: 0.8048 - accuracy: 0.5895 - val_loss: 1.7679 - val_accuracy: 0.3721
Epoch 6/10
60/60 [==============================] - 18s 304ms/step - loss: 0.9876 - accuracy: 0.4541 - val_loss: 1.3568 - val_accuracy: 0.3721
Epoch 7/10
60/60 [==============================] - 19s 316ms/step - loss: 0.9280 - accuracy: 0.4374 - val_loss: 1.4381 - val_accuracy: 0.3721

In [201]:
import tensorflow as tf
val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels))
batch_size = 128
val_dataset = val_dataset.batch(batch_size)
evaluation = model.evaluate(val_dataset)

16/16 [==============================] - 1s 77ms/step - loss: 0.8551 - accuracy: 0.3721
